In [16]:
from rdflib import Dataset, URIRef, Literal
from rdflib import Namespace
from rdflib.namespace import RDF, RDFS

In [17]:
CRM = Namespace('http://cidoc-crm.org/cidoc-crm/')

In [18]:
# the URI for our named graph that will contain our data
anno_named_graph_uri = 'http://example.org/ns/users/malte/genesis-example-geo2'
anno_named_graph_uriref = URIRef(anno_named_graph_uri)

In [19]:
# open a connection to the Blazegraph triple store
blazegraph_uri = 'http://localhost:8080/blazegraph/namespace/kb/sparql'
store = Dataset('SPARQLUpdateStore')
store.open((blazegraph_uri, blazegraph_uri))

In [20]:
# find our named graph if it exists or create a new one
anno_named_graph = None
for g in store.graphs():
    if g.identifier == anno_named_graph_uriref:
        anno_named_graph = g

if anno_named_graph is None:
    anno_named_graph = store.graph(anno_named_graph_uriref)

In [21]:
# find all annotations of type PERSON and extract the annotated name
persons = {}

res = anno_named_graph.query("""
prefix oa: <http://www.w3.org/ns/oa#>

SELECT ?anno ?name
WHERE {
  GRAPH <%s> {
    ?anno a oa:Annotation ;
      oa:hasBody / rdf:value "PERSON" ;
      oa:hasTarget / oa:hasSelector / oa:exact ?name .
  }
}
"""%(anno_named_graph_uri))

for r in res:
    name = str(r.name)
    if name in persons:
        persons[name].append(r.anno)
    else:
        persons[name] = [r.anno]

In [22]:
#persons

In [23]:
# find all annotations of type River and extract the annotated name
rivers = {}

res = anno_named_graph.query("""
prefix oa: <http://www.w3.org/ns/oa#>

SELECT ?anno ?name
WHERE {
  GRAPH <%s> {
    ?anno a oa:Annotation ;
      oa:hasBody / rdf:value "River" ;
      oa:hasTarget / oa:hasSelector / oa:exact ?name .
  }
}
"""%(anno_named_graph_uri))

for r in res:
    name = str(r.name)
    if name in rivers:
        rivers[name].append(r.anno)
    else:
        rivers[name] = [r.anno]

In [24]:
rivers

{'Pishon': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/73b5f0a0-09aa-49aa-9cfa-84de5e13e192')],
 'Gihon': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/9e2504b7-e4f3-41c5-9c19-2d72ebc9ee50')],
 'Hiddekel': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/00491e8c-0699-486e-9146-da308639d073')]}

In [25]:
# find all annotations of type Feature and extract the annotated name
features = {}

res = anno_named_graph.query("""
prefix oa: <http://www.w3.org/ns/oa#>

SELECT ?anno ?name
WHERE {
  GRAPH <%s> {
    ?anno a oa:Annotation ;
      oa:hasBody / rdf:value "PLACE" ;
      oa:hasTarget / oa:hasSelector / oa:exact ?name .
  }
}
"""%(anno_named_graph_uri))

for r in res:
    name = str(r.name)
    if name in features:
        features[name].append(r.anno)
    else:
        features[name] = [r.anno]

In [26]:
features

{'Methushael': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/a061cacc-3191-4e59-8d4b-c52ebcffc8f1')],
 'Lamech': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/437b4a2d-7083-432d-bab4-852d137541cb')],
 'Mehujael': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/f4c56c2b-9075-4efc-9337-bb16b4680be7')],
 'Havilah': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/f0e8178e-de36-4715-af4f-d45084eaa432')],
 'Arpachshad': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/d15919ed-34f4-464c-9268-5a153ac90342')],
 'Lehabim': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/d46d8d96-9026-4cde-b5c6-49f602d3579a')],
 'Pishon': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/73b5f0a0-09aa-49aa-9cfa-84de5e13e192')],
 'Shinar': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/fb7a0ada-ab35-4f43-b8f9-9b0e4fbce533')],
 'Javan': [rdflib.term.URIRef('https://recogito.pelagios.org/annotation/3e51

In [27]:
# create crm:E21_Person entries for each person
for name, annos in persons.items():
    person_uri = URIRef(f"{anno_named_graph_uri}/persons/{name}")
    # person_uri is a crm:E21_Person
    anno_named_graph.add((person_uri, RDF.type, CRM.E21_Person))
    # person has label name
    anno_named_graph.add((person_uri, RDFS.label, Literal(name)))
    
    for anno_uri in annos:
        # annotation_uri documents person_uri 
        anno_named_graph.add((anno_uri, CRM.E70_documents, person_uri))


In [28]:
# create crm:E53_Place entries for each river
for name, annos in rivers.items():
    river_uri = URIRef(f"{anno_named_graph_uri}/rivers/{name}")
    # person_uri is a crm:E21_Person
    anno_named_graph.add((river_uri, RDF.type, CRM.E53_Place))
    # person has label name
    anno_named_graph.add((river_uri, RDFS.label, Literal(name)))
    
    for anno_uri in annos:
        # annotation_uri documents person_uri 
        anno_named_graph.add((anno_uri, CRM.E70_documents, river_uri))


In [29]:
# create crm:E53_Place entries for each river
for name, annos in features.items():
    river_uri = URIRef(f"{anno_named_graph_uri}/places/{name}")
    # person_uri is a crm:E21_Person
    try:
        anno_named_graph.add((river_uri, RDF.type, CRM.E53_Place))
    except Exception as e:
        print(river_uri, e)
        pass
    # person has label name
    try:
        anno_named_graph.add((river_uri, RDFS.label, Literal(name)))
    except Exception as e: 
        print(river_uri, e)

    for anno_uri in annos:
        # annotation_uri documents person_uri 
        try:
            anno_named_graph.add((anno_uri, CRM.E70_documents, river_uri))
        except Exception as e:
            print(anno_uri, e)
            pass


In [30]:
store.close()